<a href="https://colab.research.google.com/github/Hkherdekar/Covid19/blob/master/Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install apache-airflow

In [2]:
#importing libraries
import pandas as pd #Data processing
import numpy as np #Matrix/Linear algebraic operations
import random as rd #Random number 
from datetime import datetime
from datetime import date, timedelta #date formats
import pytz

#Visualization packages
from matplotlib import pyplot as plt
import seaborn as sns

#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

from pandas import Series

#import all the required statistics functionality
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
pst = pytz.timezone('America/Los_Angeles')

In [0]:
path_to_directory = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'

In [0]:
#importing data (only last updated daily data file) and converting to pandas dataframe

date = datetime.now(pst) - timedelta(days=1)
date = date.strftime('%m-%d-%Y')
url = path_to_directory + date + '.csv'
CovidToday = pd.read_csv(url);


In [6]:
CovidToday.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-30 02:32:27,34.223334,-82.461707,29,0,0,29,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-30 02:32:27,30.295065,-92.414197,130,9,0,121,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-30 02:32:27,37.767072,-75.632346,229,4,0,225,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-30 02:32:27,43.452658,-116.241552,662,16,0,646,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-30 02:32:27,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US"


In [0]:
#importing timeseries data and converting to pandas dataframe
#The file links are hardcoded as they are auto-appended to the same file location

CovidCasesUS = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

CovidCasesGlobal = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

CovidDeathsUS = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

CovidDeathsGlobal = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

CovidRecoveredGlobal = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

  Province/State Country/Region      Lat  ...  4/27/20  4/28/20  4/29/20
0            NaN    Afghanistan  33.0000  ...     1703     1828     1939
1            NaN        Albania  41.1533  ...      736      750      766
2            NaN        Algeria  28.0339  ...     3517     3649     3848
3            NaN        Andorra  42.5063  ...      743      743      743
4            NaN         Angola -11.2027  ...       27       27       27

[5 rows x 103 columns]


  Province/State Country/Region      Lat  ...  4/27/20  4/28/20  4/29/20
0            NaN    Afghanistan  33.0000  ...       57       58       60
1            NaN        Albania  41.1533  ...       28       30       30
2            NaN        Algeria  28.0339  ...      432      437      444
3            NaN        Andorra  42.5063  ...       40       41       42
4            NaN         Angola -11.2027  ...        2        2        2

[5 rows x 103 columns]


,Province/State_left,Country/Region,Lat_left,Long_left,1/22/20_left,1/23/20_left,1/24/20_left,1/25/20_left,1/26/20_left,1/27/20_left,1/28/20_left,1/29/20_left,1/30/20_left,1/31/20_left,2/1/20_left,2/2/20_left,2/3/20_left,2/4/20_left,2/5/20_left,2/6/20_left,2/7/20_left,2/8/20_left,2/9/20_left,2/10/20_left,2/11/20_left,2/12/20_left,2/13/20_left,2/14/20_left,2/15/20_left,2/16/20_left,2/17/20_left,2/18/20_left,2/19/20_left,2/20/20_left,2/21/20_left,2/22/20_left,2/23/20_left,2/24/20_left,2/25/20_left,2/26/20_left,...,3/21/20_right,3/22/20_right,3/23/20_right,3/24/20_right,3/25/20_right,3/26/20_right,3/27/20_right,3/28/20_right,3/29/20_right,3/30/20_right,3/31/20_right,4/1/20_right,4/2/20_right,4/3/20_right,4/4/20_right,4/5/20_right,4/6/20_right,4/7/20_right,4/8/20_right,4/9/20_right,4/10/20_right,4/11/20_right,4/12/20_right,4/13/20_right,4/14/20_right,4/15/20_right,4/16/20_right,4/17/20_right,4/18/20_right,4/19/20_right,4/20/20_right,4/21/20_right,4/22/20_right,4/23/20_right,4/24/20_right,4/25/20_right,4/26/20_right,4/27/20_right,4/28/20_right,4/29/20_right
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,0,1,1,1,2,4,4,4,4,4,4,4,6,6,7,7,11,14,14,15,15,18,18,21,23,25,30,30,30,33,36,36,40,42,43,47,50,57,58,60
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,4,5,5,6,8,10,10,11,15,15,16,17,20,20,21,22,22,23,23,23,23,23,24,25,26,26,26,26,26,26,27,27,27,27,28,28,30,30
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,15,17,17,19,21,25,26,29,31,35,44,58,86,105,130,152,173,193,205,235,256,275,293,313,326,336,348,364,367,375,384,392,402,407,415,419,425,432,437,444
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,3,3,3,6,8,12,14,15,16,17,18,21,22,23,25,26,26,29,29,31,33,33,35,35,36,37,37,37,37,40,40,40,40,41,42
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
260,NaN,South Sudan,6.877000,31.307000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
#Data about coronavirus testing taken from our world in data github account to be updated daily.

CovidTesting = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv')

CovidTesting.head()

,Entity,Date,Source URL,Source label,Notes,Cumulative total,Daily change in cumulative total,Cumulative total per thousand,Daily change in cumulative total per thousand,3-day rolling mean daily change,3-day rolling mean daily change per thousand
0,Argentina - tests performed,2020-04-08,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,13330,NaN,0.295,NaN,NaN,NaN
1,Argentina - tests performed,2020-04-09,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,14850,1520.0,0.329,0.034,NaN,NaN
2,Argentina - tests performed,2020-04-10,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,16379,1529.0,0.362,0.034,NaN,NaN
3,Argentina - tests performed,2020-04-11,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,18027,1648.0,0.399,0.036,1565.667,0.035
4,Argentina - tests performed,2020-04-13,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,19758,NaN,0.437,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,Austria - tests performed,2020-04-13,https://www.sozialministerium.at/Informationen...,Austrian Ministry for Health,NaN,148412,3535.0,16.479,0.392,4556.333,0.506
96,Austria - tests performed,2020-04-14,https://www.sozialministerium.at/Informationen...,Austrian Ministry for Health,NaN,151796,3384.0,16.854,0.376,3607.000,0.400
97,Austria - tests performed,2020-04-15,https://www.sozialministerium.at/Informationen...,Austrian Ministry for Health,NaN,156801,5005.0,17.410,0.556,3974.667,0.441
98,Austria - tests performed,2020-04-16,https://www.sozialministerium.at/Informationen...,Austrian Ministry for Health,NaN,162816,6015.0,18.078,0.668,4801.333,0.533


In [9]:
#Data about population, keeping two copies for options.

WorldPopulation1 = pd.read_csv('https://raw.githubusercontent.com/Hkherdekar/Covid19/master/WPP2019_TotalPopulationBySex.csv')

WorldPopulation2 = pd.read_csv('https://raw.githubusercontent.com/Hkherdekar/Covid19/master/population-figures-by-country-csv_csv.csv')

WorldPopulation1.head()
WorldPopulation2.head()

,Country,Country_Code,Year_1960,Year_1961,Year_1962,Year_1963,Year_1964,Year_1965,Year_1966,Year_1967,Year_1968,Year_1969,Year_1970,Year_1971,Year_1972,Year_1973,Year_1974,Year_1975,Year_1976,Year_1977,Year_1978,Year_1979,Year_1980,Year_1981,Year_1982,Year_1983,Year_1984,Year_1985,Year_1986,Year_1987,Year_1988,Year_1989,Year_1990,Year_1991,Year_1992,Year_1993,Year_1994,Year_1995,Year_1996,Year_1997,Year_1998,Year_1999,Year_2000,Year_2001,Year_2002,Year_2003,Year_2004,Year_2005,Year_2006,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016
0,Aruba,ABW,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,59063.0,59440.0,59840.0,60243.0,60528.0,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,61079.0,61032.0,62149.0,64622.0,68235.0,72504.0,76700.0,80324.0,83200.0,85451.0,87277,89005,90853,92898,94992.0,97017.0,98737.0,100031.0,100832.0,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0
1,Afghanistan,AFG,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,10372630.0,10604346.0,10854428.0,11126123.0,11417825.0,11721940.0,12027822.0,12321541.0,12590286.0,12840299.0,13067538.0,13237734.0,13306695.0,13248370.0,13053954.0,12749645.0,12389269.0,12047115.0,11783050.0,11601041.0,11502761.0,11540888.0,11777609.0,12249114.0,12993657.0,13981231.0,15095099.0,16172719.0,17099541.0,17822884.0,18381605.0,18863999,19403676,20093756,20966463,21979923.0,23064851.0,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0
2,Angola,AGO,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,6309770.0,6414995.0,6523791.0,6642632.0,6776381.0,6927269.0,7094834.0,7277960.0,7474338.0,7682479.0,7900997.0,8130988.0,8376147.0,8641521.0,8929900.0,9244507.0,9582156.0,9931562.0,10277321.0,10609042.0,10921037.0,11218268.0,11513968.0,11827237.0,12171441.0,12553446.0,12968345.0,13403734.0,13841301.0,14268994.0,14682284.0,15088981.0,15504318,15949766,16440924,16983266,17572649.0,18203369.0,18865716.0,19552542.0,20262399.0,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0
3,Albania,ALB,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,3227943.0,3286542.0,3266790.0,3247039.0,3227287.0,3207536.0,3187784.0,3168033.0,3148281.0,3128530,3108778,3089027,3060173,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0
4,Andorra,AND,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,24276.0,25559.0,26892.0,28232.0,29520.0,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,50434.0,52448.0,54509.0,56671.0,58888.0,60971.0,62677.0,63850.0,64360.0,64327.0,64142,64370,65390,67341,70049.0,73182.0,76244.0,78867.0,80991.0,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0


In [21]:
#Using date value to filter data for a join 
date = datetime.now(pst) - timedelta(days=1)
date = date.strftime('%m/%d/%y')
print(date[1:])

4/29/20


In [22]:
#Breakup the data to get limited columns
print(CovidCasesGlobal.head())
print(CovidDeathsGlobal.head())

  Province/State Country/Region      Lat  ...  4/27/20  4/28/20  4/29/20
0            NaN    Afghanistan  33.0000  ...     1703     1828     1939
1            NaN        Albania  41.1533  ...      736      750      766
2            NaN        Algeria  28.0339  ...     3517     3649     3848
3            NaN        Andorra  42.5063  ...      743      743      743
4            NaN         Angola -11.2027  ...       27       27       27

[5 rows x 103 columns]
  Province/State Country/Region      Lat  ...  4/27/20  4/28/20  4/29/20
0            NaN    Afghanistan  33.0000  ...       57       58       60
1            NaN        Albania  41.1533  ...       28       30       30
2            NaN        Algeria  28.0339  ...      432      437      444
3            NaN        Andorra  42.5063  ...       40       41       42
4            NaN         Angola -11.2027  ...        2        2        2

[5 rows x 103 columns]


In [0]:
GlobalCases = CovidCasesGlobal.filter(items=['Country/Region', date[1:]])

GlobalDeaths = CovidDeathsGlobal.filter(items=['Country/Region', date[1:]])

In [26]:
GlobalCases.join(GlobalDeaths.set_index('Country/Region'), on='Country/Region', lsuffix='_cases', rsuffix='_deaths')

,Country/Region,4/29/20_cases,4/29/20_deaths
0,Afghanistan,1939,60
1,Albania,766,30
2,Algeria,3848,444
3,Andorra,743,42
4,Angola,27,2
...,...,...,...
259,France,1,0
260,South Sudan,34,0
261,Western Sahara,6,0
262,Sao Tome and Principe,8,0


In [28]:
CovidTesting.head()

,Entity,Date,Source URL,Source label,Notes,Cumulative total,Daily change in cumulative total,Cumulative total per thousand,Daily change in cumulative total per thousand,3-day rolling mean daily change,3-day rolling mean daily change per thousand
0,Argentina - tests performed,2020-04-08,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,13330,NaN,0.295,NaN,NaN,NaN
1,Argentina - tests performed,2020-04-09,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,14850,1520.0,0.329,0.034,NaN,NaN
2,Argentina - tests performed,2020-04-10,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,16379,1529.0,0.362,0.034,NaN,NaN
3,Argentina - tests performed,2020-04-11,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,18027,1648.0,0.399,0.036,1565.667,0.035
4,Argentina - tests performed,2020-04-13,https://www.argentina.gob.ar/sites/default/fil...,Government of Argentina,NaN,19758,NaN,0.437,NaN,NaN,NaN
